# Risk-targeted Ground Motion Amplitudes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bakerjw/shra-jupyter/blob/main/notebooks/09%20Risk%20Targeted%20Hazard%20Level.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.interpolate import interp1d

In [ ]:
# Define colors
color_spec = [
    [56/255, 95/255, 150/255],
    [207/255, 89/255, 33/255],
    [158/255, 184/255, 219/255],
    [231/255, 184/255, 0/255],
    [128/255, 0/255, 0/255] ]

In [3]:
# Load previous hazard curve
x = np.logspace(np.log10(0.001), np.log10(2), 100)  # IM values to consider

In [ ]:
# Mean hazard from Chapter 6 simplified logic tree
lambda_mean = np.array([0.0251694954594308, 0.0243005973346253, 0.0234365472558623, 0.0225792462468511, 0.0217305158448367,
                       0.0208920837822666, 0.0200655715142511, 0.0192524837495085, 0.0184542000865126, 0.0176719688005284,
                       0.0169069027730509, 0.0161599775046000, 0.0154320311063565, 0.0147237661269249, 0.0140357530384293,
                       0.0133684351816874, 0.0127221349535275, 0.0120970610102702, 0.0114933162595579, 0.0109109064174252,
                       0.0103497489179256, 0.00980968197780037, 0.00929047363756543, 0.00879183062195665, 0.00831340688589137,
                       0.00785481173602961, 0.00741561744179956, 0.00699536627266170, 0.00659357691982798, 0.00620975028017252,
                       0.00584337459734840, 0.00549392996997187, 0.00516089224907525, 0.00484373635688936, 0.00454193906650127,
                       0.00425498128721521, 0.00398234990373885, 0.00372353921886845, 0.00347805204941328, 0.00324540052394014,
                       0.00302510662878499, 0.00281670254590154, 0.00261973082270939, 0.00243374441035183, 0.00225830660283430,
                       0.00209299090552312, 0.00193738085754465, 0.00179106982882404, 0.00165366080890037, 0.00152476620129790,
                       0.00140400763414932, 0.00129101579497441, 0.00118543029502282, 0.00108689956639214, 0.000995080793225799,
                       0.000909639876668399, 0.000830251431896055, 0.000756598814428752, 0.000688374172055584, 0.000625278518042679,
                       0.000567021820830418, 0.000513323105142555, 0.000463910559307276, 0.000418521643610899, 0.000376903194651442,
                       0.000338811520914285, 0.000304012485139082, 0.000272281569469359, 0.000243403919858480, 0.000217174366732617,
                       0.000193397419468867, 0.000171887232821039, 0.000152467544004570, 0.000134971579723542, 0.000119241932976582,
                       0.000105130410005028, 9.24978482376873e-05, 8.12139065351012e-05, 7.11568294363794e-05, 6.22131874592354e-05,
                       5.42775957953396e-05, 4.72524139769621e-05, 4.10474292661717e-05, 3.55795266353024e-05, 3.07723482686310e-05,
                       2.65559455227739e-05, 2.28664262410104e-05, 1.96456002286791e-05, 1.68406255681954e-05, 1.44036582884222e-05,
                       1.22915077099856e-05, 1.04652995717140e-05, 8.89014880956513e-06, 7.53484361431181e-06, 6.37154214337735e-06,
                       5.37548301094735e-06, 4.52471043365022e-06, 3.79981467097529e-06, 3.18368817390031e-06, 2.66129764494389e-06])
lambda_im = lambda_mean  # Pick a hazard curve to use

In [ ]:
d_lambda = abs(np.append(np.diff(lambda_im), 0))  # Derivative of hazard curve

In [ ]:
# Find risk-targeted fragility
lambda_targ = 2e-4  # Target collapse risk
p_targ = 1 - np.exp(-lambda_targ * 50)  # Collapse probability in 50 years

beta_im = 0.6  # Assumed fragility beta
theta_im = np.arange(0.1, 0.61, 0.01)
fail_rate = np.zeros_like(theta_im)

In [ ]:
for i in range(len(theta_im)):
    p_fail = norm.cdf(np.log(x), np.log(theta_im[i]), beta_im)
    fail_rate[i] = np.sum(p_fail * d_lambda)

# Create the interpolation function
interpolator = interp1d(fail_rate, theta_im, fill_value="extrapolate")
theta_target = interpolator(lambda_targ)  # Find risk-targeted theta

In [ ]:
theta_im

In [ ]:
plt.figure()
plt.plot(theta_im, fail_rate, '-', linewidth=2, color=color_spec[0])
plt.plot(theta_im, lambda_targ * np.ones_like(theta_im), ':k', linewidth=1)
plt.plot(theta_target * np.ones(2), [0, lambda_targ], '--k', linewidth=1)
plt.legend(['λ(F | θ)', 'Target λ(F)'])
plt.xlabel('Fragility median, θ [g]')
plt.ylabel('Annual failure rate, λ(F)')
plt.xlim([0.1,0.6])
plt.ylim([0,0.0007])

In [ ]:
# Alternate beta
beta_im2 = 0.7  # Assumed fragility beta
fail_rate2 = np.zeros_like(theta_im)
for i in range(len(theta_im)):
    p_fail = norm.cdf(np.log(x), np.log(theta_im[i]), beta_im2)
    fail_rate2[i] = np.sum(p_fail * d_lambda)

# Create the interpolation function
interpolator2 = interp1d(fail_rate2, theta_im, fill_value="extrapolate")
theta_target2 = interpolator2(lambda_targ)  # Find risk-targeted theta

In [ ]:
# Another alternate beta
beta_im3 = 0.8  # Assumed fragility beta
fail_rate3 = np.zeros_like(theta_im)
for i in range(len(theta_im)):
    p_fail = norm.cdf(np.log(x), np.log(theta_im[i]), beta_im3)
    fail_rate3[i] = np.sum(p_fail * d_lambda)

# Create the interpolation function
interpolator3 = interp1d(fail_rate3, theta_im, fill_value="extrapolate")
theta_target3 = interpolator3(lambda_targ)  # Find risk-targeted theta

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(x, norm.cdf(np.log(x), np.log(theta_target), beta_im), '-', linewidth=2, color=color_spec[0])
ax1.plot(x, norm.cdf(np.log(x), np.log(theta_target2), beta_im2), '-', linewidth=2, color=color_spec[1])
ax1.plot(x, norm.cdf(np.log(x), np.log(theta_target3), beta_im3), '-', linewidth=2, color=color_spec[2])
ax1.set_ylabel('Probability of failure, P(F | SA=x)')
ax1.set_xlabel('Spectral Acceleration, SA(1 s) [g]')
ax2.plot(x, lambda_im, '--', linewidth=2, color=color_spec[3])
ax2.set_ylim([0, 0.002])
ax2.set_ylabel('Annual exceedance rate, λ(SA>x)')
fig.legend(['β = ' + str(beta_im), 'β = ' + str(beta_im2), 'β = ' + str(beta_im3), 'Ground-motion hazard'], loc='lower right', bbox_to_anchor=(0.9, 0.15))
ax1.set_xlim([0, 1])
plt.show()